<a href="https://colab.research.google.com/github/bigpine-choi/commercial-real-estate-analysis/blob/main/LSTM_%ED%88%AC%EC%9E%90%EC%88%98%EC%9D%B5%EB%A5%A0_%EC%98%88%EC%B8%A1%EC%83%81%EC%8A%B9%EB%A5%A0_%EC%A7%80%EB%8F%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📌 투자수익률 예측 상승률 기반 지도 시각화

과거 평균 수익률과 LSTM 예측 수익률을 비교하여 상승률을 계산하고, 이를 기반으로 순위를 시각화합니다.

In [3]:
import pandas as pd
import folium
import numpy as np

# ✅ 파일 경로 설정
past_file = "오피스_임대료_공실률_투자수익률_순영업소득_all.csv"
future_file = "LSTM_예측_투자수익률_지역별_all.csv"
geo_file = "서울_지역별_위도_경도.csv"

# ✅ 데이터 불러오기
past_df = pd.read_csv(past_file, encoding="utf-8-sig")
future_df = pd.read_csv(future_file, encoding="utf-8-sig")
geo_df = pd.read_csv(geo_file, encoding="utf-8-sig")

# ✅ 과거 수익률 평균 계산
past_df = past_df[past_df["지표"] == "투자수익률"]
past_avg = past_df.groupby("CLS_NM")["DTA_VAL"].mean().reset_index()
past_avg.columns = ["지역", "현재_수익률_평균"]

# ✅ 미래 수익률 평균 정리
future_df = future_df.copy()
future_df.reset_index(inplace=True)
future_df.rename(columns={"index": "지역", "예측평균": "미래_수익률_평균"}, inplace=True)

# ✅ 여기서 문자열로 변환!
past_avg["지역"] = past_avg["지역"].astype(str)
future_df["지역"] = future_df["지역"].astype(str)

# ✅ 병합 및 상승률 계산
merged = pd.merge(past_avg, future_df, on="지역", how="inner")
merged["예측_상승률"] = merged["미래_수익률_평균"] - merged["현재_수익률_평균"]

# ✅ 순위 계산
merged = merged.sort_values("예측_상승률", ascending=False).reset_index(drop=True)
merged["순위"] = merged.index + 1

# ✅ 위도/경도 병합
merged = pd.merge(merged, geo_df, on="지역", how="left")
merged = merged.dropna(subset=["위도", "경도"])

# ✅ Folium 지도 생성
seoul_center = [37.5665, 126.9780]
m = folium.Map(location=seoul_center, zoom_start=11)

for _, row in merged.iterrows():
    color = "red" if row["순위"] == 1 else "blue" if row["순위"] <= 5 else "gray"
    popup_html = (
        f"<b>지역:</b> {row['지역']}<br>"
        f"<b>현재 수익률 평균:</b> {round(row['현재_수익률_평균'], 2)}%<br>"
        f"<b>예측 수익률 평균:</b> {round(row['미래_수익률_평균'], 2)}%<br>"
        f"<b>예측 상승률:</b> {round(row['예측_상승률'], 2)}%<br>"
        f"<b>순위:</b> {row['순위']}"
    )
    folium.CircleMarker(
        location=[row["위도"], row["경도"]],
        radius=8,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=f"{row['순위']}위: {row['지역']}"
    ).add_to(m)

# ✅ 지도 저장
m.save("LSTM_투자수익률_예측상승률_지도.html")
print("✅ 지도 저장 완료!")


KeyError: '지역'